## Python type 의 등장

파이썬도 3.5 버전 부터 `typing` 이라는 모듈을 이용해서 이제 복잡한 타입을 정의할 수 있도록 해주었습니다. 다만 기본적으로 동적언어이기 때문에 runtime 시점에도 type checking 이 이뤄지는 것을 보장하지는 않습니다.

In [11]:
class Fruit():
    name: str
    color: str
    weight: float
    bazam: dict[str, list[tuple[int, bool, float]]]
    
    def __init__(self, name: str, color: str, weight: float, bazam: dict[str, list[tuple[int, bool, float]]]):
        self.name = name
        self.color = color
        self.weight = weight
        self.bazam = bazam
        

따라서 아래와 같은 방식으로 코드를 작성해도 문제없이 돌아갑니다.

In [12]:
Fruit(name=1, color=1, weight=1, bazam="bazam")

Pydantic 을 이용하는 가장 쉬운 방법은 아래와 같이 클래스를 만들고 `Pydantic` 을 상속하도록 만드는 것이다.

In [13]:
#!pip install pydantic
from typing import Annotated, Literal
from annotated_types import Gt # greater than x
from pydantic import BaseModel, Field

class Fruit(BaseModel):
    name: str
    color: Literal["red", "green"]
    weight: Annotated[float, Gt(0)]
    bazam: dict[str, list[tuple[int, bool, float]]]
    

Fruit(name="apple", color="red", weight=100, bazam={"a": [(1, True, 1.0)]})

Fruit(name='apple', color='red', weight=100.0, bazam={'a': [(1, True, 1.0)]})

In [15]:
Fruit(name=1, color=1, weight=1, bazam="bazam")

ValidationError: 3 validation errors for Fruit
name
  Input should be a valid string [type=string_type, input_value=1, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type
color
  Input should be 'red' or 'green' [type=literal_error, input_value=1, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/literal_error
bazam
  Input should be a valid dictionary [type=dict_type, input_value='bazam', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/dict_type

In [16]:
Fruit(name="banana", color="yellow", weight=100, bazam={"a": [(1, True, 1.0)]})

ValidationError: 1 validation error for Fruit
color
  Input should be 'red' or 'green' [type=literal_error, input_value='yellow', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/literal_error

공식문서에 보기 좋게 [Pydantic](https://www.youtube.com/watch?v=pWZw7hYoRVU) 이 어떻게 동작하는지 설명하고 있는데, 한번 궁금하신 분들은 보셔도 좋을듯 합니다. 

## 직렬화(Serialization)

직렬화는 객체를 byte 형태로 바꾸는 것을 뜻합니다. 우리가 보통 **API** 를 만든다고 할때, 외부에서 프로그램에 어떠한 호출을 하게되면 프로그램 내부에서 외부로 데이터가 나가야 할때가 있습니다. 
이럴때 우리가 사용하는 객체를 직렬화 해야하는데요. Pydantic 은 직렬화를 아래 세가지 방식으로 지원합니다.

- 파이썬 객체로 이루어진 `dict` 로 변환
- JSON 으로 변환 가능한 타입으로만 이루어진 `dict` 로 변환
- JSON 문자열로 변환

세가지 방식에 대해서는 사실 예제와 함께 알아보는게 쉬워서 예제와 함께 알아보도록 하겠습니다.

### 파이썬 객체로 이루어진 `dict` 로 변환

In [17]:
from pydantic import BaseModel
from datetime import datetime
from typing import Optional, List

class User(BaseModel):
    id: int
    username: str
    signup_ts: Optional[datetime] = None
    friends: List[int] = []
    is_active: bool = True

user_data = {
    "id": 123,
    "username": "pydantic_lover",
    "signup_ts": datetime(2023, 4, 15, 10, 30, 0),
    "friends": [1, 2, 3]
}
user_instance = User(**user_data)

# 방법 1: 파이썬 객체로 이루어진 dict로 변환
python_dict_objects = user_instance.model_dump()

print(f"타입: {type(python_dict_objects)}")
print(f"내용: {python_dict_objects}")

if 'signup_ts' in python_dict_objects and python_dict_objects['signup_ts'] is not None:
    print(f"signup_ts의 타입: {type(python_dict_objects['signup_ts'])}")
else:
    print("signup_ts 필드가 없거나 None입니다.")
print("-" * 30)

방법 1: model_dump() 결과 (Python 객체 유지)
타입: <class 'dict'>
내용: {'id': 123, 'username': 'pydantic_lover', 'signup_ts': datetime.datetime(2023, 4, 15, 10, 30), 'friends': [1, 2, 3], 'is_active': True}
signup_ts의 타입: <class 'datetime.datetime'>
------------------------------


코드를 보면 `model_dump()` 실행후 타입이 `dict` 타입으로 변한것을 확인할 수 있습니다. Dictionary 의 출력값을 확인해봐도 값을 그대로 보존한채 dictionary 형태로만 바뀐것을 확인할 수 있습니다.

### JSON 으로 변환 가능한 타입으로만 이루어진 `dict` 로 변환

In [19]:
from pydantic import BaseModel
from datetime import datetime
from typing import Optional, List

class User(BaseModel):
    id: int
    username: str
    signup_ts: Optional[datetime] = None
    friends: List[int] = []
    is_active: bool = True

user_data = {
    "id": 123,
    "username": "pydantic_lover",
    "signup_ts": datetime(2023, 4, 15, 10, 30, 0),
    "friends": [1, 2, 3]
}
user_instance = User(**user_data)

jsonable_dict = user_instance.model_dump(mode='json')

print(f"타입: {type(jsonable_dict)}")
print(f"내용: {jsonable_dict}")

if 'signup_ts' in jsonable_dict and jsonable_dict['signup_ts'] is not None:
    print(f"signup_ts의 타입: {type(jsonable_dict['signup_ts'])}")
else:
    print("signup_ts 필드가 없거나 None입니다.")
print("-" * 30)

방법 2: model_dump(mode='json') 결과 (JSON 호환 타입)
타입: <class 'dict'>
내용: {'id': 123, 'username': 'pydantic_lover', 'signup_ts': '2023-04-15T10:30:00', 'friends': [1, 2, 3], 'is_active': True}
signup_ts의 타입: <class 'str'>
------------------------------


살짝 햇갈리지만 잘 알아둬야 하는 부분은 `jsonable` 하다는 것입니다. 보시면 `signup_ts` 가 ISO 8601 형태로 변한것을 확인할 수 있는데요. 
이는 JSON 은 python datetime object 를 이해하지 못하기에 ISO 8601 로 변환하여 json 변환 가능한 상태로 변한것을 확인할 수 있습니다. 
한번 jsonalble 이 dump 가 잘되는지도 확인해볼까요?

In [20]:
import json

json_string = json.dumps(jsonable_dict)
print(f"JSON 문자열: {json_string}")
print("-" * 30)

JSON 문자열: {"id": 123, "username": "pydantic_lover", "signup_ts": "2023-04-15T10:30:00", "friends": [1, 2, 3], "is_active": true}
------------------------------


### JSON 문자열로 변환

In [ ]:
from pydantic import BaseModel
from datetime import datetime
from typing import Optional, List

class User(BaseModel):
    id: int
    username: str
    signup_ts: Optional[datetime] = None
    friends: List[int] = []
    is_active: bool = True

user_data = {
    "id": 123,
    "username": "pydantic_lover",
    "signup_ts": datetime(2023, 4, 15, 10, 30, 0),
    "friends": [1, 2, 3]
}
user_instance = User(**user_data)

json_string = user_instance.model_dump_json()

print(f"타입: {type(json_string)}")
print(f"내용: {json_string}")
print("-" * 30)